In [1]:
!pip install pandas

     |████████████████████████████████| 10.1MB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 512kB 16.1MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 451kB/s  eta 0:00:01     |████████████████████████████▍   | 15.4MB 29.5MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium --yes
import folium

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.0.1               |           py37_0         727 KB  conda-forge
    asn1crypto-0.24.0          |        py37_1003         154 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    cffi-1.12.3                |   py37h8022711_0         218 KB  conda-forge
    chardet-3.0.4              |        py37_1003         167 KB  conda-forge
    cryptography-2.6.1         |   py37h72c5cf5_0         607 KB  conda-forge
    folium-0.8.3               |             py_0          54 KB  conda-forge
    idna-2.8                   |        py37_1000         100 KB  conda-forge
    libblas-3.8.0              |       8_openblas           6 KB  conda-forge
    libcblas-3.8.

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv")
df.shape

(8, 56)

In [4]:
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host']]
#df = df[df['ID'].str.startswith("TS")]
df.head(10)

,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207
5,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TS1271
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201


In [5]:
def getLocation(storeName, postcode, country):
    
    if country == 'ENG': country = "UK"
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(postcode+" "+country)
    location_alt = geolocator.geocode(storeName+" "+country)
    try:
        lat = location.latitude
        lng = location.longitude
    except:
        try:
            lat = location_alt.latitude
            lng = location_alt.longitude
        except:
            lat = 0
            lng = 0
            
    return(lat,lng)

In [6]:
latitude = []
longitude = []

for storeName, postcode, country in zip(df['NAME'],df['Postcode'],df['COUNTRY']):
    lati, lngi = getLocation(storeName, postcode, country)
    latitude.append(lati)
    longitude.append(lngi)

df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude

In [7]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df

8


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,Latitude,Longitude
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001,51.515534,-0.141352
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207,54.975903,-1.612382
5,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TS1271,34.067233,-118.352108
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012,53.462559,-2.346119
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201,51.516683,-0.405498


In [24]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
categoryId = '4bf58dd8d48988d1f2941735'

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    
    store_list = []
    lat_list = []
    lng_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name 
        for i in range(0,resultsVen.shape[0]):
            store_list.append(name)
            lat_list.append(lat)
            lng_list.append(lng)

        store = pd.DataFrame([store_list,lat_list,lng_list]).T
        nearbyVenues = pd.concat([store, resultsVenCat], axis=1)

    return(nearbyVenues)

In [26]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for storeId in df['ID'].unique():
    store_info = df[df['ID']==storeId]
    regions_venues = regions_venues.append(getNearbyVenues(names=store_info['ID'],
                                                  latitudes=store_info['Latitude'],
                                                  longitudes=store_info['Longitude'],
                                                  radius=radius),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={0:'Branch ID',
                          1:'Branch Lat',
                          2:'Branch Lng',
                          'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(regions_venues.shape[0])

387


In [27]:
regions_venues

,Branch ID,Branch Lat,Branch Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,MS0325,51.5155,-0.141352,4ac518f0f964a520afad20e3,NikeTown,33,"[236 Oxford St, London, Greater London, W1C 1D...",51.515488,-0.141824,Sporting Goods Shop
1,MS0325,51.5155,-0.141352,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
2,MS0325,51.5155,-0.141352,51d86b1d498e44075e0a7840,Foot Locker,439,"[109 Oxford Street, London, Greater London, W1...",51.515890,-0.135038,Sporting Goods Shop
3,MS0325,51.5155,-0.141352,5ca5b84a9d7468003917baa1,Up & Running,2405,"[72-74 Fleet Street, London, Greater London, E...",51.514142,-0.106705,Sporting Goods Shop
4,MS0325,51.5155,-0.141352,5804fc2b38fa33f7c3387dae,New Balance,175,"[287 - 291 Oxford Street, London, Greater Lond...",51.515116,-0.143791,Sporting Goods Shop
5,MS0325,51.5155,-0.141352,4da064b3784f3704428698af,Sports Direct,246,"[150 Oxford St, London, Greater London, W1D 1N...",51.515899,-0.137837,Sporting Goods Shop
6,MS0325,51.5155,-0.141352,5ab0b6d8c5309336291c9659,Decathlon,4097,"[146-158 High Street Kensington, London, Great...",51.500470,-0.195313,Sporting Goods Shop
7,MS0325,51.5155,-0.141352,55897485498ed8423f6b80c8,Decathlon,7687,"[55-57 Garratt Lane, London, Greater London, S...",51.454381,-0.192866,Sporting Goods Shop
8,MS0325,51.5155,-0.141352,4fbf51d3e4b07c0ab040c868,JD Sports,11112,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
9,MS0325,51.5155,-0.141352,56640c11498e0520920c554a,Lululemon,5627,"[Ariel Way Unit 2099 (First Floor), City of Lo...",51.507333,-0.221508,Sporting Goods Shop


In [28]:
jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports.to_csv('jdsports_proximty.csv')
jdsports

,Branch ID,Branch Lat,Branch Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
1,MS0325,51.5155,-0.141352,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
8,MS0325,51.5155,-0.141352,4fbf51d3e4b07c0ab040c868,JD Sports,11112,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
19,MS0325,51.5155,-0.141352,5a8ecd3d5a2c9117b2eca1d4,JD Sports,9868,"[53 Tooting High Street, Tooting, Greater Lond...",51.428388,-0.167446,Sporting Goods Shop
51,TM7329,51.5155,-0.141352,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
58,TM7329,51.5155,-0.141352,4fbf51d3e4b07c0ab040c868,JD Sports,11112,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
69,TM7329,51.5155,-0.141352,5a8ecd3d5a2c9117b2eca1d4,JD Sports,9868,"[53 Tooting High Street, Tooting, Greater Lond...",51.428388,-0.167446,Sporting Goods Shop
101,TS0001,51.5155,-0.141352,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
109,TS0001,51.5155,-0.141352,4fbf51d3e4b07c0ab040c868,JD Sports,11112,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
119,TS0001,51.5155,-0.141352,5a8ecd3d5a2c9117b2eca1d4,JD Sports,9868,"[53 Tooting High Street, Tooting, Greater Lond...",51.428388,-0.167446,Sporting Goods Shop
151,TS0873,51.5141,-0.0949138,4fbf51d3e4b07c0ab040c868,JD Sports,9147,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop


In [63]:
df_map

,Branch ID,Branch Lat,Branch Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
101,TS0001,51.5155,-0.141352,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
109,TS0001,51.5155,-0.141352,4fbf51d3e4b07c0ab040c868,JD Sports,11112,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
119,TS0001,51.5155,-0.141352,5a8ecd3d5a2c9117b2eca1d4,JD Sports,9868,"[53 Tooting High Street, Tooting, Greater Lond...",51.428388,-0.167446,Sporting Goods Shop
151,TS0873,51.5141,-0.0949138,4fbf51d3e4b07c0ab040c868,JD Sports,9147,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
171,TS0873,51.5141,-0.0949138,4ac518f1f964a520ecad20e3,JD Sports,3095,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
293,TS1012,53.4626,-2.34612,4edd380cbe7be2833b142d45,JD Sports,482,"[48 Regents Crescent, Unit L12, Trafford Centr...",53.466613,-2.348693,Sporting Goods Shop
295,TS1012,53.4626,-2.34612,5a8eceff67af3a74d388fc4b,JD Sports,10761,"[Unit 23 The Birtles Sc, Wythenshawe, Greater ...",53.379649,-2.262698,Sporting Goods Shop
296,TS1012,53.4626,-2.34612,5a8ecf001acf1143546edcc7,JD Sports,3079,"[9-10 Kingsway Mall, Stretford, Greater Manche...",53.445258,-2.309869,Sporting Goods Shop
298,TS1012,53.4626,-2.34612,4b7ec528f964a52010fe2fe3,JD Sports,7233,"[16-18 Arndale Centre, Barbarolli Mall, Manche...",53.483574,-2.242812,Sporting Goods Shop
304,TS1012,53.4626,-2.34612,5a4545b8c0f16320555bd027,JD Sports,5603,"[Unit G, Regent Retail Park, Salford, M5 4DE, ...",53.477943,-2.265591,Sporting Goods Shop


In [69]:
df_map = jdsports[jdsports['Branch ID'].str.startswith("TS")]

m = folium.Map(location=[53.4626, -2.34612], zoom_start=6)
for branch_lat, branch_lng in zip(df_map['Branch Lat'], df_map['Branch Lng']):
    folium.Marker(location=[branch_lat, branch_lng], popup='Topshop').add_to(m)
    folium.Circle(location=[branch_lat, branch_lng], popup='Topshop', radius=10000, fill=False, color='blue').add_to(m)

for jd_lat, jd_lng in zip(df_map['Venue Lat'], df_map['Venue Lng']):
    folium.Circle(location=[jd_lat, jd_lng], popup='JD Sports', radius=200, fill=True, color='red').add_to(m)

m

In [ ]:
#Which TM/TS stores do not have a JD Sports withing a 2km radius?
#Add map feature with TM/TS as centre point and surrounding JD Sports.